In [ ]:
import math
import numpy as np
from src.rowwise_metrics import direct_mutual_information, euclid, cosine
from src.simulation import get_spectrum

In [ ]:
from src.simulation import get_spectra

n = 20
spectra, calibration = get_spectra(n, ['Fe', 'Al'], save=True)

In [ ]:
from libs_tools.visualization.spectral import plot_spectra

In [ ]:
plot_spectra(spectra[:5], calibration)

In [ ]:
metrics = [euclid, cosine, direct_mutual_information]
matrices = [np.zeros((n, n)) for _ in metrics]
for i in range(len(metrics)):
    for j in range(n):
        for k in range(n):
            matrices[i][j][k] = metrics[i](spectra[j], spectra[k])

In [ ]:
import plotly.express as px
px.imshow(matrices[0])

In [ ]:
px.imshow(matrices[1])

In [ ]:
px.imshow(matrices[2])

In [ ]:
labels = np.linspace(0, 100, n)
y_true = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        y_true[i][j] = abs(labels[i] - labels[j])

In [ ]:
px.imshow(y_true)

In [ ]:
px.imshow(abs((y_true / y_true.max()) - (matrices[0] / matrices[0].max())))

In [ ]:
from libs_tools.visualization.spectral import plot_spectra

matrices[0] = matrices[0] / matrices[0][0].max()
matrices[2] = matrices[2] - matrices[2][0].min()
plot_spectra([matrix[0] for matrix in matrices], labels=[foo.__name__ for foo in metrics])